In [44]:
import json
from model.transformer import TransformerModel
from torch.utils.data import Dataset
import torch
import pandas as pd
import numpy as np

In [45]:
df = pd.read_csv('openstack.csv', index_col=0, header=0)
df = df[['commit_id', 'buggy']]

In [46]:
with open('openstack_repo_processed.json', 'r', encoding='utf-8') as f, open('openstack_msg_dict2.json','r', encoding='utf-8') as f_dict:
    json_load = json.load(f)
    msg_vob_dict = json.load(f_dict)

FileNotFoundError: [Errno 2] No such file or directory: 'openstack_msg_dict2.json'

In [21]:
msg_dict = {commit['commit_id']:commit['msg'] for commit in json_load}

In [23]:
msg_dict['00057454f6609871c2cabf874876d26a6d0d0641']

['add',
 'lun',
 '#',
 'to',
 'provider_loc',
 'in',
 'nexenta',
 'driver',
 '.',
 'fix',
 'bug',
 '1088411',
 '(',
 "'",
 'can',
 'not',
 'attach',
 'volum',
 'to',
 'instanc',
 'if',
 'use',
 'nexentastor',
 'driver',
 "'",
 ')',
 '.',
 'let',
 'nexenta',
 'driver',
 'work',
 'with',
 'ani',
 'iscsi_help',
 '.',
 'change-id',
 ':',
 'ib70ed94fa472c7281de077f0f36575216741678f',
 '<',
 'desc',
 '>',
 'it',
 'look',
 'like',
 'the',
 'bug',
 '#',
 '871278',
 'is',
 'still',
 'present',
 'in',
 'nexentastor',
 'driver',
 '.',
 'i',
 "'m",
 'use',
 'ubuntu',
 '12.04.1',
 '64bit',
 'with',
 'cinder-volum',
 'packag',
 '2012.2-0ubuntu2~cloud0',
 'and',
 'if',
 'i',
 'tri',
 'to',
 'attach',
 'cinder',
 'volum',
 'to',
 'an',
 'instanc',
 ',',
 'it',
 'fail',
 'with',
 'the',
 'follow',
 'error',
 'in',
 'nova-compute.log',
 ':',
 '2012-12-05',
 '16:10:26',
 'trace',
 'error',
 ':',
 'iscsi',
 'devic',
 'not',
 'found',
 'at',
 '/dev/disk/by-path/ip-10.1.0.2:3260-iscsi-iqn.1986-03.com.sun:02

In [24]:
max_length = 0
for value in msg_dict.values():
    if max_length < len(value):
        max_length = len(value)
max_length
max_length = 512

In [25]:
for key,value in msg_dict.items():
    if len(value) < max_length:
        msg_dict[key] = value.extend(['<pad>' for i in range(max_length-len(value))])
    elif len(value) > max_length:
        # print(key)
        msg_dict[key] = value[:-(len(value)-max_length)]
for key , values in msg_dict.items():
    print(key, values)
    break

00057454f6609871c2cabf874876d26a6d0d0641 ['add', 'lun', '#', 'to', 'provider_loc', 'in', 'nexenta', 'driver', '.', 'fix', 'bug', '1088411', '(', "'", 'can', 'not', 'attach', 'volum', 'to', 'instanc', 'if', 'use', 'nexentastor', 'driver', "'", ')', '.', 'let', 'nexenta', 'driver', 'work', 'with', 'ani', 'iscsi_help', '.', 'change-id', ':', 'ib70ed94fa472c7281de077f0f36575216741678f', '<', 'desc', '>', 'it', 'look', 'like', 'the', 'bug', '#', '871278', 'is', 'still', 'present', 'in', 'nexentastor', 'driver', '.', 'i', "'m", 'use', 'ubuntu', '12.04.1', '64bit', 'with', 'cinder-volum', 'packag', '2012.2-0ubuntu2~cloud0', 'and', 'if', 'i', 'tri', 'to', 'attach', 'cinder', 'volum', 'to', 'an', 'instanc', ',', 'it', 'fail', 'with', 'the', 'follow', 'error', 'in', 'nova-compute.log', ':', '2012-12-05', '16:10:26', 'trace', 'error', ':', 'iscsi', 'devic', 'not', 'found', 'at', '/dev/disk/by-path/ip-10.1.0.2:3260-iscsi-iqn.1986-03.com.sun:02', ':', 'cinder-volume-0000001-lun-1', 'howev', 'the', 

In [27]:
special_tokens = ['<unk>', '<pad>']
keys = list(msg_vob_dict.keys())
keys[:0] = special_tokens
msg_vob_dict2 = {key:i for i, key in enumerate(keys)}
msg_vob_dict2

{'<unk>': 0,
 '<pad>': 1,
 '--': 2,
 ',': 3,
 ':': 4,
 '.': 5,
 ')': 6,
 '(': 7,
 "'": 8,
 "''": 9,
 '``': 10,
 '|': 11,
 'trace': 12,
 'file': 13,
 'line': 14,
 'thi': 15,
 '*': 16,
 '[': 17,
 '>': 18,
 'change-id': 19,
 ']': 20,
 '<': 21,
 'use': 22,
 'fix': 23,
 'instanc': 24,
 '#': 25,
 '+': 26,
 'test': 27,
 'desc': 28,
 'bug': 29,
 '-': 30,
 'error': 31,
 '=': 32,
 'return': 33,
 'call': 34,
 'api': 35,
 'add': 36,
 'nova': 37,
 'volum': 38,
 '}': 39,
 '{': 40,
 'creat': 41,
 'method': 42,
 'chang': 43,
 'imag': 44,
 'http': 45,
 "n't": 46,
 'except': 47,
 'fail': 48,
 'driver': 49,
 'delet': 50,
 'code': 51,
 'wa': 52,
 'nova.openstack.common.rpc.amqp': 53,
 'remov': 54,
 '1': 55,
 'updat': 56,
 'doe': 57,
 'none': 58,
 'set': 59,
 'name': 60,
 'network': 61,
 "'s": 62,
 'make': 63,
 'run': 64,
 '-+': 65,
 'get': 66,
 'patch': 67,
 'ha': 68,
 'nova.api.openstack': 69,
 '%': 70,
 'log': 71,
 'context': 72,
 'object': 73,
 'request': 74,
 '0': 75,
 'server': 76,
 'valu': 77,
 'sup

In [28]:
for key, value in msg_dict.items():
    print(key, value)
    break

00057454f6609871c2cabf874876d26a6d0d0641 ['add', 'lun', '#', 'to', 'provider_loc', 'in', 'nexenta', 'driver', '.', 'fix', 'bug', '1088411', '(', "'", 'can', 'not', 'attach', 'volum', 'to', 'instanc', 'if', 'use', 'nexentastor', 'driver', "'", ')', '.', 'let', 'nexenta', 'driver', 'work', 'with', 'ani', 'iscsi_help', '.', 'change-id', ':', 'ib70ed94fa472c7281de077f0f36575216741678f', '<', 'desc', '>', 'it', 'look', 'like', 'the', 'bug', '#', '871278', 'is', 'still', 'present', 'in', 'nexentastor', 'driver', '.', 'i', "'m", 'use', 'ubuntu', '12.04.1', '64bit', 'with', 'cinder-volum', 'packag', '2012.2-0ubuntu2~cloud0', 'and', 'if', 'i', 'tri', 'to', 'attach', 'cinder', 'volum', 'to', 'an', 'instanc', ',', 'it', 'fail', 'with', 'the', 'follow', 'error', 'in', 'nova-compute.log', ':', '2012-12-05', '16:10:26', 'trace', 'error', ':', 'iscsi', 'devic', 'not', 'found', 'at', '/dev/disk/by-path/ip-10.1.0.2:3260-iscsi-iqn.1986-03.com.sun:02', ':', 'cinder-volume-0000001-lun-1', 'howev', 'the', 

In [29]:
tmp = '<pad>'
if tmp == '<pad>':
    print('ok')

ok


In [37]:
pad_dict = {}
for key,value in msg_dict.items():
    word_list = []
    mask_list = []
    pad_dict[key] = []
    for word in value:
        if word == '<pad>':
            mask_list.append(1)
        else:
            mask_list.append(0)
        if word in msg_vob_dict2.keys():
            word_list.append(msg_vob_dict2[word])
        else:
            word_list.append(msg_vob_dict2['<unk>'])
    pad_dict[key].append(word_list)
    pad_dict[key].append(mask_list)
    
        

In [38]:
len(pad_dict)

12214

In [39]:
for values in pad_dict.values():
    print(len(values))
    print(values[0])
    print(values[1])
    break

2
[36, 716, 25, 0, 3256, 0, 2147, 49, 5, 23, 29, 0, 7, 8, 0, 0, 189, 38, 0, 24, 0, 22, 11984, 49, 8, 6, 5, 701, 2147, 49, 110, 0, 179, 4755, 5, 19, 4, 0, 21, 28, 18, 0, 247, 149, 0, 29, 25, 0, 0, 238, 566, 0, 11984, 49, 5, 0, 466, 22, 508, 0, 7774, 0, 1363, 653, 0, 0, 0, 0, 142, 0, 189, 136, 38, 0, 0, 24, 3, 0, 48, 0, 0, 118, 31, 0, 3514, 4, 1295, 0, 12, 31, 4, 379, 153, 0, 195, 0, 13734, 4, 0, 236, 0, 13, 13734, 4, 0, 134, 5, 0, 0, 0, 0, 10240, 1740, 13734, 4, 0, 13734, 4, 0, 1122, 110, 5, 0, 23, 0, 0, 43, 14, 4076, 0, 0, 0, 8430, 8, 4, 8, 75, 8, 39, 6, 0, 8430, 8, 4, 8, 55, 8, 39, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [40]:
print(len(msg_vob_dict2))
# with open('openstack_msg_dict3.json','w') as f:
#     json.dump(msg_vob_dict2, f)

21981


In [41]:
features = np.zeros((0, max_length))
masks = np.zeros((0, max_length))
labels = np.zeros(0, dtype=np.int64)
for key, values in pad_dict.items():
    features = np.vstack((features, np.array(values[0])))
    masks = np.vstack((masks, np.array(values[1])))
    if df.loc[df['commit_id'] == key, 'buggy'].all():
        label = 1
    else:
        label = 0
    labels = np.hstack((labels, np.array(label)))

In [42]:
labels

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [43]:
import pickle
list = []
list.append(features)
list.append(masks)
list.append(labels)
pickle.dump(list, open('test6.pkl', 'wb'))

In [ ]:
clf = pickle.load(open('test.pkl', 'rb'))

In [ ]:
features, masks, labels = clf

In [ ]:
features[0][1]

718.0

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, clf):
        features, masks, labels = clf
        self.features_values = features
        self.masks_values = masks
        self.labels = labels

    # len()を使用すると呼ばれる
    def __len__(self):
        return len(self.features_values)

    # 要素を参照すると呼ばれる関数    
    def __getitem__(self, idx):
        features_x = torch.FloatTensor(self.features_values[idx])
        masks_x = torch.FloatTensor(self.masks_values[idx])
        labels = torch.LongTensor(self.labels[idx])
        return features_x, masks_x, labels

In [ ]:
Dataset = MyDataset(clf)

In [ ]:
# features_sample, masks_sample, labels_sample = Dataset[0]
# print(features_sample, masks_sample, labels_sample)

In [ ]:
train_dataset, val_dataset = torch.utils.data.random_split(Dataset, [0.8, 0.2])


In [ ]:
batch_size = 64
from torch.utils.data import DataLoader 
train_loader = DataLoader(train_dataset, batch_size = batch_size,shuffle=True, num_workers = 8)

test_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers = 8)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
device = torch.device('cpu')

cpu


In [ ]:
# PyTorch乱数固定用

def torch_seed(seed=123):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True
    
# 乱数初期化
torch_seed()

In [ ]:
ntokens = len(msg_vob_dict)
print(ntokens)
d_model = 200
nhead = 2
d_hid = 200
nlayers = 2
dropout = 0.2
net = TransformerModel(ntokens, d_model, nhead, d_hid, nlayers, dropout).to(device)

NameError: name 'msg_vob_dict' is not defined

In [ ]:
import numpy as np
import torch
from torcheval.metrics.functional import multiclass_auroc
# 学習用関数
def fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history, test_dataset):
    
    # tqdmライブラリのインポート
    from tqdm.notebook import tqdm

    base_epochs = len(history)
    
    inputs_auc_features = np.zeros((0,512), dtype=np.int64)
    inputs_auc_features = torch.tensor(inputs_auc_features, dtype=torch.int64)
    inputs_auc_masks = np.zeros((0,512), dtype=np.int64)
    inputs_auc_masks = torch.tensor(inputs_auc_masks, dtype=torch.int64)
    labels_auc_test = np.zeros(0, dtype=np.int64)
    labels_auc_test = torch.tensor(labels_auc_test, dtype=torch.int64)
    
    for features, masks, label in test_dataset:
        inputs_auc_features = torch.vstack((inputs_auc_features, features))
        inputs_auc_masks = torch.vstack((inputs_auc_masks, masks))
        labels_auc_test = torch.hstack((labels_auc_test, label))
  
    for epoch in range(base_epochs, num_epochs+base_epochs):
        # 1エポックあたりの正解数(精度計算用)
        n_train_acc, n_val_acc = 0, 0
        # 1エポックあたりの累積損失(平均化前)
        train_loss, val_loss = 0, 0
        # 1エポックあたりのデータ累積件数
        n_train, n_test = 0, 0
       
        # print(device)
        #訓練フェーズ
        net.train()

        # for inputs, labels in tqdm(train_loader):
        for inputs, labels in train_loader:
            # 1バッチあたりのデータ件数
            train_batch_size = len(labels)
            # 1エポックあたりのデータ累積件数
            n_train += train_batch_size
    
            # GPUヘ転送
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 勾配の初期化
            optimizer.zero_grad()

            # 予測計算
            outputs = net(inputs[0], inputs[1])

            # 損失計算
            loss = criterion(outputs, labels)

            # 勾配計算
            loss.backward()

            # パラメータ修正
            optimizer.step()

            # 予測ラベル導出
            predicted = torch.max(outputs, 1)[1]

            # 平均前の損失と正解数の計算
            # lossは平均計算が行われているので平均前の損失に戻して加算
            train_loss += loss.item() * train_batch_size 
            n_train_acc += (predicted == labels).sum().item() 
            # print(n_train_acc / n_train)
        
        #予測フェーズ
        net.eval()
        with torch.no_grad():
            for inputs_test, labels_test in test_loader:
                # 1バッチあたりのデータ件数
                test_batch_size = len(labels_test)
                # 1エポックあたりのデータ累積件数
                n_test += test_batch_size

                # GPUヘ転送
                inputs_test = inputs_test.to(device)
                labels_test = labels_test.to(device)

                # 予測計算
                outputs_test = net(inputs_test[0], inputs_test[1])

                # 損失計算
                loss_test = criterion(outputs_test, labels_test)

                # 予測ラベル導出
                predicted_test = torch.max(outputs_test, 1)[1]

                #  平均前の損失と正解数の計算
                # lossは平均計算が行われているので平均前の損失に戻して加算
                val_loss +=  loss_test.item() * test_batch_size
                n_val_acc +=  (predicted_test == labels_test).sum().item()
                # print(n_val_acc, n_test)
                # print(cal_auc(outputs_test, labels_test))
            
        # 精度計算
        # print(n_train_acc, n_train)
        # print(n_val_acc, n_test)
        train_acc = n_train_acc / n_train
        val_acc = n_val_acc / n_test
        # 損失計算
        avg_train_loss = train_loss / n_train
        avg_val_loss = val_loss / n_test
        #AUC
        # auc_score = cal_auc(net, inputs_auc_features, inputs_auc_masks, labels_auc_test, device)
        
        # 結果表示
        print (f'Epoch [{(epoch+1)}/{num_epochs+base_epochs}], loss: {avg_train_loss:.5f} acc: {train_acc:.5f} val_loss: {avg_val_loss:.5f}, val_acc: {val_acc:.5f}, auc_score: {auc_score:.5f}')
        # 記録
        item = np.array([epoch+1, avg_train_loss, train_acc, avg_val_loss, val_acc, auc_score])
        history = np.vstack((history, item))

    return history
    

def cal_auc(net, inputs_auc_features, inputs_auc_masks, labels_auc_test, device):
    
    class_cnt = len(torch.unique(labels_auc_test))
    with torch.no_grad():
        inputs_auc_features = inputs_auc_features.to(device)
        inputs_auc_masks = inputs_auc_masks.to(device)
        labels_auc_test = labels_auc_test.to(device)
        outputs_test = net(inputs_auc_features, inputs_auc_masks)
        auc = multiclass_auroc(
            input = outputs_test,
            target = labels_auc_test,
            num_classes = class_cnt,
            average="macro"
        ).item()
    
    return auc

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
from tqdm.notebook import tqdm
from IPython.display import display
from sklearn.model_selection import train_test_split

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [ ]:
import torch.nn as nn
import torch.optim as optim
history = np.zeros((0,6))
# 学習率
lr = 0.001
# 繰り返し回数
num_epochs = 20
# net._initialize_weights()
# 最適化関数: 勾配降下法
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=0.0001)
# optimizer = torch.optim.SGD(net.parameters(), lr=lr)
# 損失関数： 交差エントロピー関数
criterion = nn.CrossEntropyLoss()
history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history, val_dataset)

In [ ]:
with torch.no_grad():
    for features_test, masks_test, labels_test in test_loader:
        # 1バッチあたりのデータ件数
        test_batch_size = len(labels_test)
        # 1エポックあたりのデータ累積件数
        n_test += test_batch_size

        # GPUヘ転送
        features_test = features_test.to(device)
        masks_test = masks_test.to(device)
        labels_test = labels_test.to(device)

        # 予測計算
        outputs_test = net(features_test, masks_test)
        
        # 損失計算
        loss_test = criterion(outputs_test, labels_test)
        print(loss_test)

        # 予測ラベル導出
        predicted_test = torch.max(outputs_test, 1)[1]

        #  平均前の損失と正解数の計算
        # lossは平均計算が行われているので平均前の損失に戻して加算
        val_loss +=  loss_test.item() * test_batch_size
        n_val_acc +=  (predicted_test == labels_test).sum().item()
        print(val_loss, n_val_acc)